In [4]:
print("OK!")

OK!


In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Pinecone
from langchain_pinecone import Pinecone 
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\jerry\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
from dotenv import load_dotenv, find_dotenv 
load_dotenv()
#load_dotenv(find_dotenv()) # if above doesn't work. use this one. 

True

In [7]:
import os 

PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV=os.getenv("PINECONE_API_ENV")

In [8]:
# create an index in pinecone.  dim 384


In [9]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [20]:
extracted_data = load_pdf("data")

In [28]:
#check data 
extracted_data[100:110]

[Document(page_content='Normal results\nIf the surgery is successful, the ovaries will be\nremoved without complication, and the underlying prob-lem resolved. In the case of cancer, all the cancer will beremoved.\nAbnormal results\nComplications may arise if the surgeon finds that\ncancer has spread to other places in the abdomen. If thecancer cannot be removed by surgery, it must be treatedwith chemotherapy and radiation.\nResources\nPERIODICALS\n“Removing Ovaries Lowers Risk for Women at High Risk of\nBreast, Ovarian Cancer.” ACS News Today 8 November\n2000. <http://www.cancer.org>.\nORGANIZATIONS\nAmerican Cancer Society National Headquarters. 1599 Clifton\nRoad NE, Atlanta, GA 30329. (800)ACS-2345. <http://www.cancer.org>.\nCancer Information Service, National Cancer Institute. Build-\ning 31, Room 10A19, 9000 Rockville Pike, Bethesda, MD20892. (800)4-CANCER. <http://www.nci.nih.gov/cancerinfo/index.html>.\nTish Davidson, A.M.\nOpen fracture reduction seeFracture repair\nOphthalmic

In [30]:
# Create text chunks 
def text_split(extracted_data) : 
  return  RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20).split_documents(extracted_data)


In [31]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 24487


In [33]:
# text_chunks
text_chunks[1000:1010]

[Document(page_content='though, like all major surgery, it does carry some risks.These include unanticipated reaction to anesthesia, inter-nal bleeding, blood clots, accidental damage to otherorgans, and post-surgery infection.\nComplications after an oophorectomy include', metadata={'source': 'data\\Gale Encyclopedia of Medicine v4N_S.pdf', 'page': 99}),
 Document(page_content='changes in sex drive, hot flashes, and other symptoms ofmenopause if both ovaries are removed. Women whohave both ovaries removed and who do not take estrogenreplacement therapy run an increased risk for cardiovas-cular disease and osteoporosis. Women with a history ofpsychological and emotional problems before anoophorectomy are more likely to experience psychologi-cal difficulties after the operation.\nGALE ENCYCLOPEDIA OF MEDICINE 2 2393Oophorectomy', metadata={'source': 'data\\Gale Encyclopedia of Medicine v4N_S.pdf', 'page': 99}),
 Document(page_content='Normal results\nIf the surgery is successful, the ov

In [34]:
#download embedding model

model_name="sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)


c:\Users\jerry\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
c:\Users\jerry\anaconda3\envs\mchatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [36]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [47]:
#query_result
#Initializing the Pinecone

pc=pinecone.Pinecone(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV,
              regionnnnn="us-east-1")


In [63]:
index_name="medical-chatbot"

index = pc.Index(index_name)

In [66]:
#!pip install langchain_pinecone 

In [67]:

from langchain_pinecone import Pinecone   # from langgchain
#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


In [83]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"
query = "Why sometime I can feel ichy"


docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='In autonomic dysreflexia, irritation of the skin,\nbowel, or bladder causes a highly exaggerated responsefrom autonomic nerves. This response is caused by theuncontrolled release of norepinephrine, a hormone simi-lar to adrenaline. Uncontrolled release of norepinephrinecauses a rapid rise in blood pressure and a slowing of theheart rate. These symptoms are accompanied by throb-bing headache , nausea, anxiety , sweating, and goose'), Document(page_content='head (fontanel) may be depressed. Symptoms of dehy-\ndration at any age include cracked lips, dry or sticky\nmouth, lethargy, and sunken eyes. A person who is\ndehydrated cries without shedding tears and does not\nurinate very often. The skin is less elastic than it should\nbe and is slow to return to its normal position after\nbeing pinched.\nDehydration can cause confusion, constipation , dis-\ncomfort, drowsiness, fever, and thirst. The skin turns pale'), Document(page_content='bing pain caused by irr

In [84]:
for d in docs: print(d.page_content); print("\n -- EOL -- \n")

In autonomic dysreflexia, irritation of the skin,
bowel, or bladder causes a highly exaggerated responsefrom autonomic nerves. This response is caused by theuncontrolled release of norepinephrine, a hormone simi-lar to adrenaline. Uncontrolled release of norepinephrinecauses a rapid rise in blood pressure and a slowing of theheart rate. These symptoms are accompanied by throb-bing headache , nausea, anxiety , sweating, and goose

 -- EOL -- 

head (fontanel) may be depressed. Symptoms of dehy-
dration at any age include cracked lips, dry or sticky
mouth, lethargy, and sunken eyes. A person who is
dehydrated cries without shedding tears and does not
urinate very often. The skin is less elastic than it should
be and is slow to return to its normal position after
being pinched.
Dehydration can cause confusion, constipation , dis-
comfort, drowsiness, fever, and thirst. The skin turns pale

 -- EOL -- 

bing pain caused by irritation of or damage to a nerve.
The pain is usually brief but m

In [85]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [86]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [99]:
from langchain.llms import CTransformers
import transformers

In [100]:

m="Suriya03/llama-2-7b-chat.ggmlv3.q5_1.bin"  # No model file found in repo 
m="sonyjamalpur/llama-2-7b-chat.ggmlv3.q8_0.bin" # No model file found in repo 
m="patliu1001/llama-2-7b-chat.ggmlv3.q8_0.bin"
m="model/llama-2-7b-chat.ggmlv3.q4_0.bin"   # not found
m="Tripti-123/llama-2-7b-chat.ggmlv3.q8_0.bin"
m="TheBloke/Llama-2-7B-Chat-GGML"
llm=CTransformers(model=m, model_type="llama", config={'max_new_tokens':512,'temperature':0.01})

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


RuntimeError: Failed to create LLM 'llama' from 'C:\Users\jerry\.cache\huggingface\hub\models--TheBloke--Llama-2-7B-Chat-GGML\snapshots\76cd63c351ae389e1d4b91cab2cf470aab11864b\llama-2-7b-chat.ggmlv3.q2_K.bin'.

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


KeyboardInterrupt: 